# Transformación de datos

## Resumen

El código carga y combina varios archivos CSV, luego transforma estos datos en varias tablas pivote basadas en diferentes características de los productos (`producto`, `familia`, `categoria`, `subcategoria`). Después de las transformaciones, los resultados se guardan en nuevos archivos CSV para su posterior análisis o uso.

In [2]:
# Bibliotecas requeridas
import sys
sys.path.append('../scr/')

import loadAndTransformData as ltd
import viewDataFunctions as vdf

import pandas as pd
import os
import random

In [3]:
# Carga de datos
# data = ltd.cargar_csv('C:/Users/jpmog/Downloads/tu_(1)_2024_08_31.csv')
data = ltd.combinar_csv_por_prefijo('../data/raw/', 'Minimarket_Sales_Data')

In [4]:
data.shape

(152781, 11)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152781 entries, 0 to 152780
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   fecha                 152781 non-null  object 
 1   codigo_factura        152781 non-null  int64  
 2   mnt_total_a_pagar     152781 non-null  float64
 3   cant_items            152781 non-null  int64  
 4   familia               152524 non-null  object 
 5   categoria             152781 non-null  object 
 6   subcategoria          152781 non-null  object 
 7   codigo_producto       152781 non-null  float64
 8   cantidad_vendida      152781 non-null  float64
 9   venta_bruta_producto  152781 non-null  float64
 10  venta_neta_producto   152781 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 12.8+ MB


In [6]:
# Modificar a tipo object las columnas codigo_factura, codigo_producto
data = ltd.convert_columns_to_object(data, ['codigo_factura', 'codigo_producto'])

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152781 entries, 0 to 152780
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   fecha                 152781 non-null  object 
 1   codigo_factura        152781 non-null  object 
 2   mnt_total_a_pagar     152781 non-null  float64
 3   cant_items            152781 non-null  int64  
 4   familia               152524 non-null  object 
 5   categoria             152781 non-null  object 
 6   subcategoria          152781 non-null  object 
 7   codigo_producto       152781 non-null  object 
 8   cantidad_vendida      152781 non-null  float64
 9   venta_bruta_producto  152781 non-null  float64
 10  venta_neta_producto   152781 non-null  float64
dtypes: float64(4), int64(1), object(6)
memory usage: 12.8+ MB


In [8]:
# Modificar a tipo object las columnas codigo_factura, codigo_producto
data = ltd.convert_columns_to_category(data, ['codigo_factura', 'codigo_producto', 'familia', 'categoria', 'subcategoria'])
data['fecha'] = pd.to_datetime(data['fecha'])

In [9]:
#  Obtener el dataframe agrupado
df_agrupado = ltd.procesar_y_agrupar(data, '../data/temp/chunks/', ['fecha', 'codigo_factura', 'mnt_total_a_pagar', 'cant_items', 
                                                                'codigo_producto', 'familia', 'categoria', 'subcategoria'], 
                                 ['cantidad_vendida', 'venta_bruta_producto', 'venta_neta_producto'], 'sum', 
                                 chunksize_guardado=10**6, chunksize_lectura=10**5)

In [10]:
# Excluir facturas anuladas
data_filtrada = df_agrupado[df_agrupado['mnt_total_a_pagar']>0]

In [11]:
data_filtrada

,fecha,codigo_factura,mnt_total_a_pagar,cant_items,codigo_producto,familia,categoria,subcategoria,cantidad_vendida,venta_bruta_producto,venta_neta_producto
0,2024-01-01,2324741063,7540.0,4,12000043.0,Commodities,Granos,Otros,3.0,7350.0,7350.0
1,2024-01-01,2324741063,7540.0,4,12000162.0,Empaques y Bazar,Otros,Empaques,1.0,190.0,104.0
2,2024-01-01,2324741641,4990.0,1,12000178.0,Aseo,Aseo Hogar,Detergentes,1.0,4990.0,4193.0
3,2024-01-01,2324749607,16870.0,3,12000325.0,Bebidas,Lácteos,UHT Personal,1.0,7990.0,6714.0
4,2024-01-01,2324749607,16870.0,3,12001261.0,Refrigerados,Lácteos,Combinados,1.0,2090.0,1756.0
...,...,...,...,...,...,...,...,...,...,...,...
52487,2024-03-01,2545426767,15790.0,3,12000176.0,Aseo,Hogar,Tissue,1.0,4350.0,3655.0
52488,2024-03-01,2545426767,15790.0,3,12000376.0,Bebidas,Bebidas Líquidas,Gasificadas,1.0,2750.0,2311.0
52489,2024-03-01,2545426767,15790.0,3,12002501.0,Alimentos Dulces,Pasabocas,Frutos Secos,1.0,8690.0,7303.0
52490,2024-03-01,2545461183,8180.0,2,12001129.0,Mascotas,Mascotas,Alimento Seco,1.0,5490.0,5229.0


In [12]:
model_data_producto = ltd.pivotear_df(data_filtrada, 'codigo_factura', 'codigo_producto')
model_data_familia = ltd.pivotear_df(data_filtrada, 'codigo_factura', 'familia')
model_data_categoria = ltd.pivotear_df(data_filtrada, 'codigo_factura', 'categoria')
model_data_subcategoria = ltd.pivotear_df(data_filtrada, 'codigo_factura', 'subcategoria')

In [13]:
ltd.guardar_csv(model_data_producto, '../data/processed/', 'Minimarket_Sales_Data_Transform_By_Product')
ltd.guardar_csv(model_data_familia, '../data/processed/', 'Minimarket_Sales_Data_Transform_By_Family')
ltd.guardar_csv(model_data_categoria, '../data/processed/', 'Minimarket_Sales_Data_Transform_By_Category')
ltd.guardar_csv(model_data_subcategoria, '../data/processed/', 'Minimarket_Sales_Data_Transform_By_Subcategory')

Archivo guardado exitosamente en: ../data/processed/Minimarket_Sales_Data_Transform_By_Product.csv
Archivo guardado exitosamente en: ../data/processed/Minimarket_Sales_Data_Transform_By_Family.csv
Archivo guardado exitosamente en: ../data/processed/Minimarket_Sales_Data_Transform_By_Category.csv
Archivo guardado exitosamente en: ../data/processed/Minimarket_Sales_Data_Transform_By_Subcategory.csv


## Descripción del procedimiento

Este código realiza varias operaciones de transformación y procesamiento de datos en un conjunto de archivos CSV que contienen información de ventas de un minimercado. A continuación se describe el código paso a paso:

### Importación de Bibliotecas y Módulos
1. **`sys.path.append('../scr/')`**: Agrega un directorio específico (`'../scr/'`) al `sys.path`, lo que permite importar módulos desde esa ruta.
2. **`import loadAndTransformData as ltd`**: Importa un módulo personalizado `loadAndTransformData` con el alias `ltd`, que probablemente contiene funciones para cargar y transformar datos.
3. **`import viewDataFunctions as vdf`**: Importa otro módulo personalizado `viewDataFunctions` con el alias `vdf`, que posiblemente incluye funciones para visualizar datos.
4. **`import pandas as pd` y `import os`**: Importa las bibliotecas estándar `pandas` para la manipulación de datos y `os` para operaciones relacionadas con el sistema de archivos.

### Carga de Datos
5. **`data = ltd.combinar_csv_por_prefijo('../data/raw/', 'Minimarket_Sales_Data')`**: Utiliza una función del módulo `ltd` para combinar múltiples archivos CSV que comparten un prefijo común (`'Minimarket_Sales_Data'`) desde el directorio `'../data/raw/'`. Esto produce un DataFrame llamado `data`.

6. **`data.shape` y `data.info()`**: Imprime las dimensiones del DataFrame (`data.shape`) y muestra un resumen del contenido del DataFrame (`data.info()`), incluyendo el tipo de datos y la cantidad de valores no nulos en cada columna.

### Transformación de Datos
7. **`data = ltd.convert_columns_to_category(data, ['codigo_factura', 'codigo_producto', 'familia', 'categoria', 'subcategoria'])`**: Ajusta el tipo de datos de las columnas indicadas a categoria. 
8. **`df_agrupado = ltd.procesar_y_agrupar(...)`**: Agrupa el conjunto de datos por las columnas 'fecha', 'codigo_factura', 'mnt_total_a_pagar', 'cant_items', 'codigo_producto', 'familia', 'categoria', 'subcategoria'; y suma las columnas 'cantidad_vendida', 'venta_bruta_producto', 'venta_neta_producto'.
9. **`data_filtrada = df_agrupado[df_agrupado['mnt_total_a_pagar']>0]`**: Excluye las facturas con monto negativo que corresponden a facturas anuladas y por tanto no son ventas reales. 
7. **`model_data_producto = ltd.pivotear_df(data, 'codigo_factura', 'codigo_producto')`**: Crea una tabla pivote del DataFrame `data`, donde las filas están indexadas por `'codigo_factura'` y las columnas por `'codigo_producto'`. El resultado se guarda en `model_data_producto`.
8. **`model_data_familia = ltd.pivotear_df(data, 'codigo_factura', 'familia')`**: Similar al paso anterior, pero pivoteando por la columna `'familia'`, guardando el resultado en `model_data_familia`.
9. **`model_data_categoria = ltd.pivotear_df(data, 'codigo_factura', 'categoria')`**: Crea una tabla pivote basada en la columna `'categoria'` y la guarda en `model_data_categoria`.
10. **`model_data_subcategoria = ltd.pivotear_df(data, 'codigo_factura', 'subcategoria')`**: Crea otra tabla pivote, esta vez basada en `'subcategoria'`, guardando el resultado en `model_data_subcategoria`.
11. **`print(model_data_producto.shape, model_data_familia.shape, model_data_categoria.shape, model_data_subcategoria.shape)`**: Imprime las dimensiones de los cuatro DataFrames resultantes de las operaciones de pivoteo.

### Guardado de Datos Transformados
15. **`ltd.guardar_csv(model_data_producto, '../data/processed/', 'Minimarket_Sales_Data_Transform_By_Product')`**: Guarda el DataFrame `model_data_producto` como un archivo CSV en el directorio `'../data/processed/'` con el nombre `'Minimarket_Sales_Data_Transform_By_Product'`.

13. **`ltd.guardar_csv(model_data_familia, '../data/processed/', 'Minimarket_Sales_Data_Transform_By_Family')`**: Similar al paso anterior, pero guardando `model_data_familia` con un nombre de archivo diferente.

14. **`ltd.guardar_csv(model_data_categoria, '../data/processed/', 'Minimarket_Sales_Data_Transform_By_Category')`**: Guarda `model_data_categoria` en un archivo CSV.

15. **`ltd.guardar_csv(model_data_subcategoria, '../data/processed/', 'Minimarket_Sales_Data_Transform_By_Subcategory')`**: Finalmente, guarda `model_data_subcategoria` como un archivo CSV.